# Uniquely Identifying Particles

In many cases, one can just identify particles by their position in the particle array, e.g. using ``sim.particles[5]``. However, in cases where particles might get reordered in the particle array finding a particle might be difficult. This is why we added a *hash* attribute to particles.

In REBOUND particles might get rearranged when a tree code is used for the gravity or collision routine, when particles merge, when a particle leave the simulation box, or when you manually remove or add particles. For these cases, it is useful to keep track of which particle is which.

In this example, we show the basic usage of the *hash* attribute. We start by creating a simulation with 3 particles:

In [1]:
import rebound
sim = rebound.Simulation()
sim.add(m=1.)
sim.add(a=0.32)
sim.add(a=1.)
sim.add(a=2.)

We can refer to a particle using a custom string as follows:

In [2]:
sim.particles[1].hash = sim.get_particle_hash("mercury")
sim.particles[2].hash = sim.get_particle_hash("earth")

REBOUND uses a hash function to convert "earth" to an effectively random integer, and we store that identifier in the particle's hash field.  We can now access the particle with:

In [3]:
print(sim.get_particle("earth"))

<rebound.Particle object, m=0.0 x=1.0 y=0.0 z=0.0 vx=0.0 vy=1.0 vz=0.0>


Of course, we can still access the particle in the standard way:

In [4]:
print(sim.particles[2])

<rebound.Particle object, m=0.0 x=1.0 y=0.0 z=0.0 vx=0.0 vy=1.0 vz=0.0>


We can remove the ``mercury`` particle with 

In [5]:
sim.remove(name="mercury")

If we try to access the ``earth`` particle in the same way as above, we get a different result because the particle moved from index 2 to index 1:

In [6]:
print(sim.get_particle("earth"))
print(sim.particles[2])

<rebound.Particle object, m=0.0 x=1.0 y=0.0 z=0.0 vx=0.0 vy=1.0 vz=0.0>
<rebound.Particle object, m=0.0 x=2.0 y=0.0 z=0.0 vx=0.0 vy=0.707106781187 vz=0.0>


For convenience you can also add the name when you add the particle to the simulation:

In [7]:
sim.add(a=1000., name="Planet 9")
print(sim.get_particle("Planet 9").a)

1000.0


If you need to assign many hashes and don't want to give them each a name, you can also let the simulation assign them unique hashes by not passing a string to `sim.get_particle_hash` after the fact:

In [8]:
for i in range(3,13):
    sim.add(a=i)
    sim.particles[-1].hash = sim.get_particle_hash()

If we print all the particles' hashes, ones that were not assigned one are 0, ones assigned strings will have random integers, and ones assigned by the simulation will be consecutive (plus a random offset):

In [9]:
for p in sim.particles:
    print(p.hash)

0
1424801690
0
545758163
13679
13680
13681
13682
13683
13684
13685
13686
13687
13688


We can also access particles through their hash:

In [10]:
print(sim.get_particle(hash=1424801690))

<rebound.Particle object, m=0.0 x=1.0 y=0.0 z=0.0 vx=0.0 vy=1.0 vz=0.0>


Only particles that have been assigned a non-zero hash can be accessed with `sim.get_particle`.  If you assign hashes manually and two particles share the same hash, `sim.get_particle` will return the first hit in the `particles` array.